In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
import pandas as pd
import numpy as np
import os
import time
import re
import unidecode
import time 
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
pwd

'C:\\Users\\soyma\\Documents\\GitHub\\Precandidaturas-ERM-2022\\rop'

### Partidos políticos

In [20]:
driver = webdriver.Chrome( ChromeDriverManager().install() )
driver.maximize_window()

url = 'https://aplicaciones007.jne.gob.pe/srop_publico/Consulta/OrganizacionPolitica'
driver.get( url )



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\soyma\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [22]:
# Store all_tables
all_tables = {}

In [23]:
for dpt_idx in range( 1 ):
    
    # Get again all departments since HTML is refreshing
    # all elements
    # Click on one specific department
    dpt = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[1]/select" ) )
    department = dpt.options[ dpt_idx ]
    
    # Get departmant name
    dpt_name = department.text
    
    # We select a different department name
    if dpt_name == "Partido Político" :
        
        # click on department
        department.click()
        
        time.sleep(3)
        
        # Get all elements of province
        prov = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[2]/select" ) )
        num_prov_options = len( prov.options )
        
        for prov_idx in range( num_prov_options ):
            
            # Get again all districts since HTML is refreshing
            # all elements
            prov = Select( driver.find_element_by_xpath( "/html/body/div[1]/div[2]/div[1]/div/div[2]/select" ) )
            province = prov.options[ prov_idx ]
                
            # Get province name
            prov_name = province.text
            
            if prov_name != "--TODOS--" :
                
                # click on province
                province.click()
                
                driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[3]/input').click()
                        
                time.sleep(10)
                
                link = driver.current_url
                
                table_path = driver.find_element_by_xpath( "/html/body/div[1]/div[3]" )
                table_html = table_path.get_attribute( 'innerHTML' )
                table = pd.read_html( table_html )
                        
                table_clean = table[0].iloc[ 0:, [0,1,2]].copy()
                                
                table_clean[ 'Organizacion' ] = dpt_name
                table_clean[ 'Estado' ]   = prov_name
                
                all_tables[ province ] = table_clean

In [24]:
final_data = pd.concat( all_tables.values() ).reset_index( drop = True )

In [25]:
final_data.to_excel( 'partido_politico.xlsx' , index = False )

In [26]:
driver.quit()